##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Lite Model Maker によるテキスト検索

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/text_searcher"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/models/modify/model_maker/text_searcher.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/models/modify/model_maker/text_searcher.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a>
</td>
  <td> ノートブックをダウンロード</td>
  <td>     <a href="https://tfhub.dev/google/universal-sentence-encoder-lite/2"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">TF Hub モデルを参照</a>
</td>
</table>

この colab ノートブックでは、[TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/models/modify/model_maker) ライブラリを使用して、TFLite Searcher モデルを作成する方法について説明します。テキスト検索モデルを使用すると、アプリでセマンティック検索またはスマートリプライを構築できます。この種類のモデルでは、テキストクエリを取得して、Web ページのデータベースなどのテキストデータセットにおける最も関連性が高いエントリを検索できます。このモデルは、URL、ページタイトル、他のテキスト入力識別子などの指定されたメタデータを含む、データセットの最小距離スコアエントリのリストを返します。構築した後は、[Task Library Searcher API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/text_searcher) を使用してデバイス (例: Android) にデプロイし、数行のコードだけで推論を実行できます。

このチュートリアルでは、CNN/DailyMail データセットをインスタンスとして利用し、TFLite 検索モデルを作成します。互換性がある入力カンマ区切り値 (CSV) 形式で独自のデータセットを試すことができます。

## 拡張最近傍を使用したテキスト検索

このチュートリアルでは、[GitHub repo](https://github.com/abisee/cnn-dailymail) から生成された、公開 CNN/DailyMail 非匿名化集約データセットを使用します。このデータセットには、30 万件以上の新しい記事が含まれており、検索モデルを構築するための良質のデータセットになります。また、テキストクエリのモデル推論中に、さまざまな関連ニュースを返します。

この例のテキスト検索モデルでは、[ScaNN](https://github.com/google-research/google-research/tree/master/scann) (Scalable Nearest Neighbors: 拡張最近傍) インデックスファイルを使用します。このファイルは、定義済みのデータベースから類似した項目を検索できます。ScaNN では、最先端のパフォーマンスが実現され、大規模なベクトル類似性検索を効率的に実行できます。

この colab では、このデータセットのハイライトと URL が使用され、モデルを作成します。

1. ハイライトは、埋め込み特徴量ベクトルを生成するためのテキストであり、検索で使用されます。
2. URL は、関連するハイライトを検索した後に、ユーザーに表示される返された結果です。

このチュートリアルでは、このデータを CSV ファイルに保存し、その CSV ファイルを使用してモデルを構築します。次に、データセットの例をいくつか示します。

ハイライト | URL
--- | ---
Hawaiian Airlines again lands at No. 1 in on-time performance. The Airline Quality Rankings Report looks at the 14 largest U.S. airlines. ExpressJet <br> and American Airlines had the worst on-time performance. Virgin America had the best baggage  handling; Southwest had lowest complaint rate. | http://www.cnn.com/2013/04/08/travel/airline-quality-report
European football's governing body reveals list of countries bidding to host 2020 finals. The 60th anniversary edition of the finals will be hosted by 13 <br> countries. Thirty-two countries are considering bids to host 2020 matches. UEFA will announce host cities on September 25. | http://edition.cnn.com:80/2013/09/20/sport/football/football-euro-2020-bid-countries/index.html?
Once octopus-hunter Dylan Mayer has now also signed a petition of 5,000 divers banning their hunt at Seacrest Park. Decision by Washington <br> Department of Fish and Wildlife could take months. | http://www.dailymail.co.uk:80/news/article-2238423/Dylan-Mayer-Washington-considers-ban-Octopus-hunting-diver-caught-ate-Puget-Sound.html?
Galaxy was observed 420 million years after the Big Bang. found by NASA’s Hubble Space Telescope, Spitzer Space Telescope, and one of nature’s <br> own natural 'zoom lenses' in space. | http://www.dailymail.co.uk/sciencetech/article-2233883/The-furthest-object-seen-Record-breaking-image-shows-galaxy-13-3-BILLION-light-years-Earth.html


## 設定


まず、[GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) の Model Maker パッケージなどの必要なパッケージをインストールします。

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q tflite-model-maker
!pip install gdown

必要なパッケージをインポートします。

In [ ]:
from tflite_model_maker import searcher

### データセットを準備する

このチュートリアルでは、[GitHub repo](https://github.com/abisee/cnn-dailymail) の CNN / Daily Mail 集約データセットを使用します。

まず、CNN と Daily Mail のテキストと URL をダウンロードして解凍します。Google Drive からダウンロードできなかった場合は、数分間待ってからもう一度ダウンロードするか、手動でダウンロードして colab にアップロードしてください。

In [ ]:
!gdown https://drive.google.com/uc?id=0BwmD_VLjROrfTHk4NFg2SndKcjQ
!gdown https://drive.google.com/uc?id=0BwmD_VLjROrfM1BxdkxVaTY2bWs

!wget -O all_train.txt https://raw.githubusercontent.com/abisee/cnn-dailymail/master/url_lists/all_train.txt
!tar xzf cnn_stories.tgz
!tar xzf dailymail_stories.tgz

次に、`tflite_model_maker` ライブラリにアップロードできる CSV ファイルにデータを保存します。コードは、[`tensorflow_datasets`](https://github.com/tensorflow/datasets/blob/master/tensorflow_datasets/summarization/cnn_dailymail.py) でこのデータを読み込むために使用されるロジックに基づいています。`tensorflow_dataset` には、この colab で使用されている URL が含まれていないため、直接使用することはできません。

データセット全体では、データを埋め込み特徴量ベクトルに処理するのに時間がかかるため、既定では、CNN および Daily Mail データセットの最初の 5% のみが既定で選択されます。この比率を調整するか、検索対象の CNN および Daily Mail データセットの 50% の記事が含まれている構築済みの TFLite [モデル](https://storage.googleapis.com/download.tensorflow.org/models/tflite_support/searcher/text_to_image_blogpost/cnn_daily_text_searcher.tflite) で試すことができます。

In [ ]:
#@title Save the highlights and urls to the CSV file
#@markdown Load the highlights from the stories of CNN / Daily Mail, map urls with highlights, and save them to the CSV file.

CNN_FRACTION = 0.05 #@param {type:"number"}
DAILYMAIL_FRACTION = 0.05 #@param {type:"number"}

import csv
import hashlib
import os
import tensorflow as tf

dm_single_close_quote = u"\u2019"  # unicode
dm_double_close_quote = u"\u201d"
END_TOKENS = [
    ".", "!", "?", "...", "'", "`", '"', dm_single_close_quote,
    dm_double_close_quote, ")"
]  # acceptable ways to end a sentence


def read_file(file_path):
  """Reads lines in the file."""
  lines = []
  with tf.io.gfile.GFile(file_path, "r") as f:
    for line in f:
      lines.append(line.strip())
  return lines


def url_hash(url):
  """Gets the hash value of the url."""
  h = hashlib.sha1()
  url = url.encode("utf-8")
  h.update(url)
  return h.hexdigest()


def get_url_hashes_dict(urls_path):
  """Gets hashes dict that maps the hash value to the original url in file."""
  urls = read_file(urls_path)
  return {url_hash(url): url[url.find("id_/") + 4:] for url in urls}


def find_files(folder, url_dict):
  """Finds files corresponding to the urls in the folder."""
  all_files = tf.io.gfile.listdir(folder)
  ret_files = []
  for file in all_files:
    # Gets the file name without extension.
    filename = os.path.splitext(os.path.basename(file))[0]
    if filename in url_dict:
      ret_files.append(os.path.join(folder, file))
  return ret_files


def fix_missing_period(line):
  """Adds a period to a line that is missing a period."""
  if "@highlight" in line:
    return line
  if not line:
    return line
  if line[-1] in END_TOKENS:
    return line
  return line + "."


def get_highlights(story_file):
  """Gets highlights from a story file path."""
  lines = read_file(story_file)

  # Put periods on the ends of lines that are missing them
  # (this is a problem in the dataset because many image captions don't end in
  # periods; consequently they end up in the body of the article as run-on
  # sentences)
  lines = [fix_missing_period(line) for line in lines]

  # Separate out article and abstract sentences
  highlight_list = []
  next_is_highlight = False
  for line in lines:
    if not line:
      continue  # empty line
    elif line.startswith("@highlight"):
      next_is_highlight = True
    elif next_is_highlight:
      highlight_list.append(line)

  # Make highlights into a single string.
  highlights = "\n".join(highlight_list)

  return highlights

url_hashes_dict = get_url_hashes_dict("all_train.txt")
cnn_files = find_files("cnn/stories", url_hashes_dict)
dailymail_files = find_files("dailymail/stories", url_hashes_dict)

# The size to be selected.
cnn_size = int(CNN_FRACTION * len(cnn_files))
dailymail_size = int(DAILYMAIL_FRACTION * len(dailymail_files))
print("CNN size: %d"%cnn_size)
print("Daily Mail size: %d"%dailymail_size)

with open("cnn_dailymail.csv", "w") as csvfile:
  writer = csv.DictWriter(csvfile, fieldnames=["highlights", "urls"])
  writer.writeheader()

  for file in cnn_files[:cnn_size] + dailymail_files[:dailymail_size]:
    highlights = get_highlights(file)
    # Gets the filename which is the hash value of the url.
    filename = os.path.splitext(os.path.basename(file))[0]
    url = url_hashes_dict[filename]
    writer.writerow({"highlights": highlights, "urls": url})


## テキスト検索モデルの構築

テキスト検索モデルを作成するには、データセットを読み込み、そのデータを使用してモデルを作成し、TFLite モデルをエクスポートします。

### ステップ 1. データセットの読み込み

Model Maker では、CSV 形式のテキストデータセットと、各テキスト文字列 (この例の URL など) の対応するメタデータが取り込まれます。ユーザーが指定した埋め込みモデルを使用して、テキスト文字列が特徴量ベクトルに埋め込まれます。

このデモでは、[Universal Sentence Encoder](https://tfhub.dev/google/universal-sentence-encoder-lite/2) を使用して検索モデルを構築します。これは、すでに [colab](https://github.com/tensorflow/tflite-support/blob/master/tensorflow_lite_support/examples/colab/on_device_text_to_image_search_tflite.ipynb) から再トレーニングされた最先端の文章埋め込みモデルです。このモデルは、オンデバイス推論パフォーマンスのために最適化され、クエリ文字列を埋め込む時間はわずか 6 ミリ秒です (Pixel 6 で測定)。あるいは、小さいサイズながらも各埋め込み時間が 38 ミリ秒である、[この](https://tfhub.dev/google/lite-model/universal-sentence-encoder-qa-ondevice/1?lite-format=tflite)量子化バージョンを使用することもできます。

In [ ]:
!wget -O universal_sentence_encoder.tflite https://storage.googleapis.com/download.tensorflow.org/models/tflite_support/searcher/text_to_image_blogpost/text_embedder.tflite

`searcher.TextDataLoader` インスタンスを作成し、`data_loader.load_from_csv` メソッドを使用して、データセットを読み込みます。このステップでは、各テキストの埋め込み特徴量ベクトルが 1 つずつ生成されるため、最長で 10 分間かかります。独自の CSV ファイルをアップロードし、それを読み込んで、カスタマイズされたモデルを構築することもできます。

CSV ファイルでテキスト列とメタデータ列の名前を指定します。

- テキストは、埋め込み特徴量ベクトルを生成するために使用されます。
- メタデータは、特定のテキストを検索するときに表示されるコンテンツです。

次に、上記で生成された CNN-DailyMail CSV ファイルの最初の 4 行を示します。

ハイライト | URL
--- | ---
Syrian official: Obama climbed to the top of the tree, doesn't know how to get down. Obama sends a letter to the heads of the House and Senate. Obama <br> to seek congressional approval on military action against Syria. Aim is to determine whether CW were used, not by whom, says U.N. spokesman. | http://www.cnn.com/2013/08/31/world/meast/syria-civil-war/
Usain Bolt wins third gold of world championship. Anchors Jamaica to 4x100m relay victory. Eighth gold at the championships for Bolt. Jamaica double <br> up in women's 4x100m relay. | http://edition.cnn.com/2013/08/18/sport/athletics-bolt-jamaica-gold
The employee in agency's Kansas City office is among hundreds of "virtual" workers. The employee's travel to and from the mainland U.S. last year cost <br> more than $24,000. The telecommuting program, like all GSA practices, is under review. | http://www.cnn.com:80/2012/08/23/politics/gsa-hawaii-teleworking
NEW: A Canadian doctor says she was part of a team examining Harry Burkhart in 2010. NEW: Diagnosis: "autism, severe anxiety, post-traumatic stress <br> disorder and depression" Burkhart is also suspected in a German arson probe, officials say. Prosecutors believe the German national set a string of fires <br> in Los Angeles. | http://edition.cnn.com:80/2012/01/05/justice/california-arson/index.html?


In [ ]:
data_loader = searcher.TextDataLoader.create("universal_sentence_encoder.tflite", l2_normalize=True)
data_loader.load_from_csv("cnn_dailymail.csv", text_column="highlights", metadata_column="urls")

画像のユースケースでは、`searcher.ImageDataLoader` インスタンスを作成し、`data_loader.load_from_folder` を使用して、フォルダから画像を読み込むことができます。TFLite 埋め込みモデルでは、`searcher.ImageDataLoader` インスタンスを作成する必要があります。これは、クエリを特徴量ベクトルにエンコードするために使用され、TFLite 検索モデルでエクスポートされるためです。

```python
data_loader = searcher.ImageDataLoader.create("mobilenet_v2_035_96_embedder_with_metadata.tflite")
data_loader.load_from_folder("food/")
```

###ステップ 2. 検索モデルの作成

- ScaNN オプションを構成します。詳細については、[api ドキュメント](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/searcher/ScaNNOptions)を参照してください。
- データと ScaNN オプションから検索モデルを作成します。ScaNN アルゴリズムの詳細については、[in-depth examination](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html) を参照してください。

In [ ]:
scann_options = searcher.ScaNNOptions(
      distance_measure="dot_product",
      tree=searcher.Tree(num_leaves=140, num_leaves_to_search=4),
      score_ah=searcher.ScoreAH(dimensions_per_block=1, anisotropic_quantization_threshold=0.2))
model = searcher.Searcher.create_from_data(data_loader, scann_options)

上記の例では、次のオプションを定義します。

- `distance_measure`: "dot_product" を使用して、2 つの埋め込みベクトル間の距離を測定します。実際には、**負**のドット積値を計算するため、「小さいほど近い」という概念が守られます。

- `tree`: データセットは、140 のパーティション (おおよそデータサイズの平方根) に分割され、そのうちの 4 つが検索されます。これはデータセットの約 3% です。

- `score_ah`: 浮動小数点数埋め込みを同じ次元の int8 値に量子化し。スペースを削減します。

###ステップ 3. TFLite モデルのエクスポート

次に、TFLite Searcher モデルをエクスポートできます。

In [ ]:
model.export(
      export_filename="searcher.tflite",
      userinfo="",
      export_format=searcher.ExportFormat.TFLITE)

## クエリでの TFLite モデルのテスト

カスタムクエリテキストを使用して、エクスポート済みの TFLite モデルをテストできます。検索モデルを使用してテキストを照会するには、次のように、モデルを初期化して、テキストフレーズで検索を実行します。

In [ ]:
from tflite_support.task import text

# Initializes a TextSearcher object.
searcher = text.TextSearcher.create_from_file("searcher.tflite")

# Searches the input query.
results = searcher.search("The Airline Quality Rankings Report looks at the 14 largest U.S. airlines.")
print(results)

モデルをさまざまなプラットフォームに統合する方法については、[Task Library ドキュメント](https://www.tensorflow.org/lite/inference_with_metadata/task_library/text_searcher)を参照してください。

# その他の資料

詳細については、次のドキュメントを参照してください。

- TensorFlow Lite Model Maker の[ガイド](https://www.tensorflow.org/lite/models/modify/model_maker)と [API リファレンス](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker)

- タスクライブラリ: デプロイ用の [TextSearcher](https://www.tensorflow.org/lite/inference_with_metadata/task_library/text_searcher)

- エンドツーエンドリファレンスアプリ: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/text_searcher/android) 版
